In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder
import matplotlib.pyplot as plt
from pprint import pprint
import gc
from collections import deque
from tqdm.notebook import tqdm
from time import sleep
import pickle
from datetime import datetime
import msgpack

In [2]:
train = pd.read_csv('LSTM_Embedded_Train_Data_V2.csv')
features = pd.read_csv('LSTM_Features.csv')

In [3]:
train

,shop_id,item_id,item_category_id,date_block_num,year,month,avg_item_price,avg_item_price_per_shop,avg_item_price_per_month,avg_item_price_per_shop_per_month,...,shop_embedding_16,shop_embedding_17,shop_embedding_18,shop_embedding_19,shop_embedding_20,shop_embedding_21,shop_embedding_22,shop_embedding_23,shop_embedding_24,shop_embedding_25
0,2,27,19,0,2013,1,1461.228571,1498.500000,2159.125000,2499.000000,...,-0.519515,0.589719,-1.259106,0.977608,2.524584,-0.084711,0.614360,-0.218721,1.054061,0.344803
1,2,1409,19,0,2013,1,1058.565455,807.310000,1270.085714,1398.500000,...,-0.519515,0.589719,-1.259106,0.977608,2.524584,-0.084711,0.614360,-0.218721,1.054061,0.344803
2,2,1467,19,0,2013,1,957.239055,932.361111,963.444711,899.000000,...,-0.519515,0.589719,-1.259106,0.977608,2.524584,-0.084711,0.614360,-0.218721,1.054061,0.344803
3,2,1471,19,0,2013,1,1661.040967,1463.705882,2064.657252,1819.000000,...,-0.519515,0.589719,-1.259106,0.977608,2.524584,-0.084711,0.614360,-0.218721,1.054061,0.344803
4,2,1832,19,0,2013,1,939.034265,849.348519,1446.971688,1265.833333,...,-0.519515,0.589719,-1.259106,0.977608,2.524584,-0.084711,0.614360,-0.218721,1.054061,0.344803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14242867,55,13093,36,32,2015,9,250.000000,250.000000,250.000000,250.000000,...,0.622923,0.186599,-0.440730,0.843658,2.167138,-0.101304,1.226746,0.373412,0.844474,-0.231308
14242868,55,13095,36,32,2015,9,500.000000,500.000000,500.000000,500.000000,...,0.622923,0.186599,-0.440730,0.843658,2.167138,-0.101304,1.226746,0.373412,0.844474,-0.231308
14242869,55,13091,36,32,2015,9,1000.000000,1000.000000,1000.000000,1000.000000,...,0.622923,0.186599,-0.440730,0.843658,2.167138,-0.101304,1.226746,0.373412,0.844474,-0.231308
14242870,55,13094,36,32,2015,9,2500.000000,2500.000000,2500.000000,2500.000000,...,0.622923,0.186599,-0.440730,0.843658,2.167138,-0.101304,1.226746,0.373412,0.844474,-0.231308


# Forgot to Add item id embedding

In [6]:
item_id_embedding = features.groupby(['item_id'])['item_cnt_month'].mean().reset_index()
item_id_embedding.rename(columns={'item_cnt_month':'item_id_embedding'},inplace=True)


In [7]:
features = pd.merge(features, item_id_embedding, on=['item_id'])

In [8]:
features

,shop_id,item_id,item_category_id,date_block_num,year,month,avg_item_price,avg_item_price_per_shop,avg_item_price_per_month,avg_item_price_per_shop_per_month,avg_num_item_sales,avg_monthly_item_num,avg_item_sales_per_shop,item_cnt_month,item_id_embedding
0,2,27,19,0,2013,1,1461.228571,1498.5,2159.125000,2499.0,1.0,1.0,1.0,1.0,1.0
1,2,27,19,17,2014,6,1461.228571,1498.5,858.000000,498.0,1.0,1.0,1.0,1.0,1.0
2,11,27,19,0,2013,1,1461.228571,1496.5,2159.125000,1890.0,1.0,1.0,1.0,1.0,1.0
3,11,27,19,9,2013,10,1461.228571,1496.5,942.266667,1103.0,1.0,1.0,1.0,1.0,1.0
4,19,27,19,0,2013,1,1461.228571,1965.0,2159.125000,2499.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609117,55,863,78,29,2015,6,216.000000,216.0,216.000000,216.0,1.0,1.0,1.0,1.0,1.0
1609118,55,2435,31,29,2015,6,199.000000,199.0,199.000000,199.0,1.0,1.0,1.0,1.0,1.0
1609119,55,3004,76,29,2015,6,1872.000000,1872.0,1872.000000,1872.0,1.0,1.0,1.0,1.0,1.0
1609120,55,5373,76,29,2015,6,12576.000000,12576.0,12576.000000,12576.0,2.0,2.0,2.0,2.0,2.0


In [9]:
item_id_embedding

,item_id,item_id_embedding
0,0,1.000000
1,1,1.200000
2,2,1.000000
3,3,1.000000
4,4,1.000000
...,...,...
21801,22165,1.000000
21802,22166,13.920000
21803,22167,2.946429
21804,22168,1.000000


In [10]:
train = pd.merge(train, item_id_embedding, on=['item_id'])

In [12]:
with open("LSTM_Embedded_Train_Data.msgpack","wb") as f:
    packed = msgpack.packb(train)
    f.write(packed)

TypeError: can not serialize 'DataFrame' object

In [16]:
train.to_pickle('LSTM_Embedded_Train_Data.pkl')

OSError: [Errno 22] Invalid argument

In [17]:
features.to_pickle('LSTM_features.pkl')

In [18]:
len(train)

14242872

In [19]:
shop_ids = features.shop_id.unique()
train_grp = train.groupby(['shop_id'])
for shop_id in shop_ids:
    val = train_grp.get_group(shop_id)
    val.to_pickle('processed_data/LSTM_Embedded_Train_Shop_'+str(shop_id)+'.pkl')